In [ ]:
import difflib
from waste_data import RuleStore

FUZZY_CUTOFF = 0.8

# ── utility ----------------------------------------------------------
def _norm(t: str) -> str:
    return "".join(c.lower() for c in t if c.isalnum() or c.isspace()).strip()

# ── classifier -------------------------------------------------------
class WasteClassifier:
    def __init__(self):
        self.store = RuleStore()

    def classify(self, raw: str):
        term = _norm(raw)
        term = self.store.synonym_for(term)

        # 1) exact
        found = self.store.get(term)
        if found:
            cat, act = found
            return cat, act

        # 2) fuzzy
        candidates = difflib.get_close_matches(term, self.store.rules.keys(),
                                               n=1, cutoff=FUZZY_CUTOFF)
        if candidates:
            cat, act = self.store.rules[candidates[0]]
            return cat, act

        # 3) unknown
        return None, None

# ── CLI-ish helper functions ----------------------------------------
def demo_examples(clf: WasteClassifier):
    print("=== REQUIRED EXAMPLES ===")
    for it in ["banana peel", "plastic bottle", "battery"]:
        cat, act = clf.classify(it)
        print(f"{it:<20} → {cat}, {act}")

def custom_examples(clf: WasteClassifier):
    print("\n=== CUSTOM EXAMPLES ===")
    for it in ["glass jar", "tin can", "coated paper"]:
        cat, act = clf.classify(it)
        print(f"{it:<20} → {cat or 'Unknown'}, {act or 'Unknown'}")

def evaluation(clf: WasteClassifier):
    print("\n=== QUICK EVALUATION ===")
    TEST = {
        "banana peel"   : "Organic",
        "plastic bottle": "Plastic",
        "battery"       : "Hazardous",
        "glass jar"     : "Glass",
        "egg shell"     : "Organic",
        "can"           : "Metal",
        "unknown item"  : None
    }
    clf.store.evaluate(TEST, clf.classify)

def add_and_test_rule(clf: WasteClassifier):
    print("\nAdding rule: coated paper → Paper, Recycle")
    clf.store.add_rule("coated paper", "Paper", "Recycle")
    print("Check:", clf.classify("coated paper"))

# ── run everything when executed standalone --------------------------
if __name__ == "__main__":
    clf = WasteClassifier()
    demo_examples(clf)
    custom_examples(clf)
    evaluation(clf)
    add_and_test_rule(clf)